In [1]:
import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed,Interval
import datetime 
from astral.moon import phase 



# Extrayendo data de tradingview con tvdatafeed

In [2]:
username = 'labradoradrian770'
password = 'eva210920'

In [3]:
tv = TvDatafeed(username=username,password=password)

In [4]:
df_ethbtc=tv.get_hist('ETHBTC','BINANCE', Interval.in_daily,n_bars=5000)

In [5]:
df_ethbtc.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2735 entries, 2015-08-06 21:00:00 to 2023-01-30 21:00:00
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   symbol  2735 non-null   object 
 1   open    2735 non-null   float64
 2   high    2735 non-null   float64
 3   low     2735 non-null   float64
 4   close   2735 non-null   float64
 5   volume  2735 non-null   float64
dtypes: float64(5), object(1)
memory usage: 149.6+ KB


In [6]:
df_ethbtc

,symbol,open,high,low,close,volume
datetime,,,,,,
2015-08-06 21:00:00,BINANCE:ETHBTC,0.010141,0.012575,0.009262,0.009990,0.0000
2015-08-07 21:00:00,BINANCE:ETHBTC,0.009990,0.010009,0.002725,0.003079,0.0000
2015-08-08 21:00:00,BINANCE:ETHBTC,0.003079,0.003361,0.002411,0.002818,0.0000
2015-08-09 21:00:00,BINANCE:ETHBTC,0.002818,0.002818,0.002459,0.002665,0.0000
2015-08-10 21:00:00,BINANCE:ETHBTC,0.002665,0.004204,0.002521,0.003900,0.0000
...,...,...,...,...,...,...
2023-01-26 21:00:00,BINANCE:ETHBTC,0.069597,0.069670,0.068352,0.069241,83961.5377
2023-01-27 21:00:00,BINANCE:ETHBTC,0.069240,0.069426,0.067866,0.068290,53689.8892
2023-01-28 21:00:00,BINANCE:ETHBTC,0.068287,0.069712,0.068196,0.069281,82374.1648


data= df_ethbtc
data['datetime']= data.index


data

# cambiando el Datetimeindex por un index numerico incremental

In [8]:
data['datetime'] = pd.to_datetime(data['datetime'], format = "%Y-%m-%d")
data['datetime'].info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 2735 entries, 2015-08-06 21:00:00 to 2023-01-30 21:00:00
Series name: datetime
Non-Null Count  Dtype         
--------------  -----         
2735 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 42.7 KB


In [9]:
data = data.reset_index(drop=True)
data



,symbol,open,high,low,close,volume,datetime
0,BINANCE:ETHBTC,0.010141,0.012575,0.009262,0.009990,0.0000,2015-08-06 21:00:00
1,BINANCE:ETHBTC,0.009990,0.010009,0.002725,0.003079,0.0000,2015-08-07 21:00:00
2,BINANCE:ETHBTC,0.003079,0.003361,0.002411,0.002818,0.0000,2015-08-08 21:00:00
3,BINANCE:ETHBTC,0.002818,0.002818,0.002459,0.002665,0.0000,2015-08-09 21:00:00
4,BINANCE:ETHBTC,0.002665,0.004204,0.002521,0.003900,0.0000,2015-08-10 21:00:00
...,...,...,...,...,...,...,...
2730,BINANCE:ETHBTC,0.069597,0.069670,0.068352,0.069241,83961.5377,2023-01-26 21:00:00
2731,BINANCE:ETHBTC,0.069240,0.069426,0.067866,0.068290,53689.8892,2023-01-27 21:00:00
2732,BINANCE:ETHBTC,0.068287,0.069712,0.068196,0.069281,82374.1648,2023-01-28 21:00:00
2733,BINANCE:ETHBTC,0.069280,0.069340,0.067801,0.068616,82836.6726,2023-01-29 21:00:00


In [10]:
data= data.drop(columns='symbol',axis=1)
data

,open,high,low,close,volume,datetime
0,0.010141,0.012575,0.009262,0.009990,0.0000,2015-08-06 21:00:00
1,0.009990,0.010009,0.002725,0.003079,0.0000,2015-08-07 21:00:00
2,0.003079,0.003361,0.002411,0.002818,0.0000,2015-08-08 21:00:00
3,0.002818,0.002818,0.002459,0.002665,0.0000,2015-08-09 21:00:00
4,0.002665,0.004204,0.002521,0.003900,0.0000,2015-08-10 21:00:00
...,...,...,...,...,...,...
2730,0.069597,0.069670,0.068352,0.069241,83961.5377,2023-01-26 21:00:00
2731,0.069240,0.069426,0.067866,0.068290,53689.8892,2023-01-27 21:00:00
2732,0.068287,0.069712,0.068196,0.069281,82374.1648,2023-01-28 21:00:00
2733,0.069280,0.069340,0.067801,0.068616,82836.6726,2023-01-29 21:00:00


In [11]:
data['datetime'] = data['datetime'].dt.date


In [12]:
data

,open,high,low,close,volume,datetime
0,0.010141,0.012575,0.009262,0.009990,0.0000,2015-08-06
1,0.009990,0.010009,0.002725,0.003079,0.0000,2015-08-07
2,0.003079,0.003361,0.002411,0.002818,0.0000,2015-08-08
3,0.002818,0.002818,0.002459,0.002665,0.0000,2015-08-09
4,0.002665,0.004204,0.002521,0.003900,0.0000,2015-08-10
...,...,...,...,...,...,...
2730,0.069597,0.069670,0.068352,0.069241,83961.5377,2023-01-26
2731,0.069240,0.069426,0.067866,0.068290,53689.8892,2023-01-27
2732,0.068287,0.069712,0.068196,0.069281,82374.1648,2023-01-28
2733,0.069280,0.069340,0.067801,0.068616,82836.6726,2023-01-29


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2735 entries, 0 to 2734
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   open      2735 non-null   float64
 1   high      2735 non-null   float64
 2   low       2735 non-null   float64
 3   close     2735 non-null   float64
 4   volume    2735 non-null   float64
 5   datetime  2735 non-null   object 
dtypes: float64(5), object(1)
memory usage: 128.3+ KB


In [14]:
data['datetime']=pd.to_datetime(data['datetime'])
data['datetime'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2735 entries, 0 to 2734
Series name: datetime
Non-Null Count  Dtype         
--------------  -----         
2735 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 21.5 KB


In [15]:
data['moon_phase']=''
data

,open,high,low,close,volume,datetime,moon_phase
0,0.010141,0.012575,0.009262,0.009990,0.0000,2015-08-06,
1,0.009990,0.010009,0.002725,0.003079,0.0000,2015-08-07,
2,0.003079,0.003361,0.002411,0.002818,0.0000,2015-08-08,
3,0.002818,0.002818,0.002459,0.002665,0.0000,2015-08-09,
4,0.002665,0.004204,0.002521,0.003900,0.0000,2015-08-10,
...,...,...,...,...,...,...,...
2730,0.069597,0.069670,0.068352,0.069241,83961.5377,2023-01-26,
2731,0.069240,0.069426,0.067866,0.068290,53689.8892,2023-01-27,
2732,0.068287,0.069712,0.068196,0.069281,82374.1648,2023-01-28,
2733,0.069280,0.069340,0.067801,0.068616,82836.6726,2023-01-29,


# Ciclo de moon_phase de cada fecha contenida en el df

In [16]:
for index, row in data.iterrows():
    date = row["datetime"]
    phase_moon = phase(date)
    data.at[index, "moon_phase"] = phase_moon

data

,open,high,low,close,volume,datetime,moon_phase
0,0.010141,0.012575,0.009262,0.009990,0.0000,2015-08-06,20.333444
1,0.009990,0.010009,0.002725,0.003079,0.0000,2015-08-07,21.344556
2,0.003079,0.003361,0.002411,0.002818,0.0000,2015-08-08,22.355667
3,0.002818,0.002818,0.002459,0.002665,0.0000,2015-08-09,23.366778
4,0.002665,0.004204,0.002521,0.003900,0.0000,2015-08-10,24.300111
...,...,...,...,...,...,...,...
2730,0.069597,0.069670,0.068352,0.069241,83961.5377,2023-01-26,4.855667
2731,0.069240,0.069426,0.067866,0.068290,53689.8892,2023-01-27,5.866778
2732,0.068287,0.069712,0.068196,0.069281,82374.1648,2023-01-28,6.877889
2733,0.069280,0.069340,0.067801,0.068616,82836.6726,2023-01-29,7.811222


# Creando name_phase column 


In [17]:
def imprimir_fase(phase):
    if phase >=0 and phase <7:
        return "🌑 new moon"
    elif phase >= 7 and phase <14 :
        return "🌓 first quarter"
    elif phase >=14 and phase <21:
        return "🌕 full moon"
    elif phase >=21 and phase <28 :
        return "🌗 last quarter"
    else:
        return "Fase lunar desconocida."
    
    
data["name_phase"] = ""

for index, row in data.iterrows():
    phase = row["moon_phase"]
    name_phase = imprimir_fase(phase)
    data.at[index, "name_phase"] = name_phase
    
data

,open,high,low,close,volume,datetime,moon_phase,name_phase
0,0.010141,0.012575,0.009262,0.009990,0.0000,2015-08-06,20.333444,🌕 full moon
1,0.009990,0.010009,0.002725,0.003079,0.0000,2015-08-07,21.344556,🌗 last quarter
2,0.003079,0.003361,0.002411,0.002818,0.0000,2015-08-08,22.355667,🌗 last quarter
3,0.002818,0.002818,0.002459,0.002665,0.0000,2015-08-09,23.366778,🌗 last quarter
4,0.002665,0.004204,0.002521,0.003900,0.0000,2015-08-10,24.300111,🌗 last quarter
...,...,...,...,...,...,...,...,...
2730,0.069597,0.069670,0.068352,0.069241,83961.5377,2023-01-26,4.855667,🌑 new moon
2731,0.069240,0.069426,0.067866,0.068290,53689.8892,2023-01-27,5.866778,🌑 new moon
2732,0.068287,0.069712,0.068196,0.069281,82374.1648,2023-01-28,6.877889,🌑 new moon
2733,0.069280,0.069340,0.067801,0.068616,82836.6726,2023-01-29,7.811222,🌓 first quarter


# Creando change column para observar cambios de tendencia y porcentaje de cambio entre los precios de cierre

In [18]:
data.insert(data.columns.get_loc('close')+1, 'change',  data['close'].pct_change().mul(100).round(2))

In [19]:
data

,open,high,low,close,change,volume,datetime,moon_phase,name_phase
0,0.010141,0.012575,0.009262,0.009990,NaN,0.0000,2015-08-06,20.333444,🌕 full moon
1,0.009990,0.010009,0.002725,0.003079,-69.18,0.0000,2015-08-07,21.344556,🌗 last quarter
2,0.003079,0.003361,0.002411,0.002818,-8.47,0.0000,2015-08-08,22.355667,🌗 last quarter
3,0.002818,0.002818,0.002459,0.002665,-5.44,0.0000,2015-08-09,23.366778,🌗 last quarter
4,0.002665,0.004204,0.002521,0.003900,46.34,0.0000,2015-08-10,24.300111,🌗 last quarter
...,...,...,...,...,...,...,...,...,...
2730,0.069597,0.069670,0.068352,0.069241,-0.51,83961.5377,2023-01-26,4.855667,🌑 new moon
2731,0.069240,0.069426,0.067866,0.068290,-1.37,53689.8892,2023-01-27,5.866778,🌑 new moon
2732,0.068287,0.069712,0.068196,0.069281,1.45,82374.1648,2023-01-28,6.877889,🌑 new moon
2733,0.069280,0.069340,0.067801,0.068616,-0.96,82836.6726,2023-01-29,7.811222,🌓 first quarter


In [20]:
data.columns

Index(['open', 'high', 'low', 'close', 'change', 'volume', 'datetime',
       'moon_phase', 'name_phase'],
      dtype='object')

# funtion simple para volatility_index

In [30]:
def volatilidad_index(row):
    return (row['high'] - row['low']) / (row['close'] - row['open'])

data['volatility_index'] = data.apply(volatilidad_index, axis=1)
data['volatility_index'] = (data['volatility_index'] - data['volatility_index'].min()) / (data['volatility_index'].max() - data['volatility_index'].min()) * 2 - 1
data['volatility_index'] = data['volatility_index'].round(2)
data


,open,high,low,close,change,volume,datetime,moon_phase,name_phase,volatility_index
0,0.010141,0.012575,0.009262,0.009990,NaN,0.0000,2015-08-06,20.333444,🌕 full moon,0.63
1,0.009990,0.010009,0.002725,0.003079,-69.18,0.0000,2015-08-07,21.344556,🌗 last quarter,0.66
2,0.003079,0.003361,0.002411,0.002818,-8.47,0.0000,2015-08-08,22.355667,🌗 last quarter,0.65
3,0.002818,0.002818,0.002459,0.002665,-5.44,0.0000,2015-08-09,23.366778,🌗 last quarter,0.65
4,0.002665,0.004204,0.002521,0.003900,46.34,0.0000,2015-08-10,24.300111,🌗 last quarter,0.66
...,...,...,...,...,...,...,...,...,...,...
2730,0.069597,0.069670,0.068352,0.069241,-0.51,83961.5377,2023-01-26,4.855667,🌑 new moon,0.65
2731,0.069240,0.069426,0.067866,0.068290,-1.37,53689.8892,2023-01-27,5.866778,🌑 new moon,0.65
2732,0.068287,0.069712,0.068196,0.069281,1.45,82374.1648,2023-01-28,6.877889,🌑 new moon,0.66
2733,0.069280,0.069340,0.067801,0.068616,-0.96,82836.6726,2023-01-29,7.811222,🌓 first quarter,0.65


# Valores con de volatilidad mas comunes 

In [22]:
data['volatility_index'] = data['volatility_index'].astype(float)
grupo_vola = data['volatility_index'].groupby(data['volatility_index']).count()
grupo_vola

volatility_index
-1.00       1
-0.46       1
-0.24       1
 0.15       1
 0.33       1
 0.34       1
 0.44       2
 0.45       1
 0.46       2
 0.48       2
 0.49       1
 0.50       1
 0.51       1
 0.52       1
 0.54       2
 0.56       1
 0.57       1
 0.59       4
 0.60       5
 0.61       5
 0.62      12
 0.63      25
 0.64      66
 0.65    1209
 0.66    1208
 0.67     101
 0.68      32
 0.69      19
 0.70       6
 0.71       2
 0.72       1
 0.73       4
 0.75       1
 0.76       2
 0.78       3
 0.79       1
 0.80       1
 0.82       1
 0.83       2
 0.84       1
 0.95       1
 0.97       1
 1.00       1
Name: volatility_index, dtype: int64

# Aplicando describe veo que aun falta transformar los datos de las columns moon_phase y volume


In [25]:
data.describe()

,open,high,low,close,change,volume,volatility_index
count,2735.000000,2735.000000,2735.000000,2735.000000,2734.000000,2.735000e+03,2735.000000
mean,0.042943,0.044046,0.041815,0.042964,0.206167,1.361357e+05,0.653664
std,0.026699,0.027359,0.025978,0.026695,5.147682,1.436559e+05,0.048560
min,0.001613,0.001793,0.001574,0.001613,-69.180000,0.000000e+00,-1.000000
25%,0.021527,0.021935,0.021007,0.021535,-1.807500,0.000000e+00,0.650000
50%,0.033774,0.034500,0.032999,0.033825,-0.200000,1.086523e+05,0.660000
75%,0.068698,0.070071,0.067364,0.068723,1.570000,1.990891e+05,0.660000
max,0.150920,0.156357,0.148407,0.150920,49.460000,1.396957e+06,1.000000
